In [1]:
from dotenv import load_dotenv

load_dotenv()

True

# Defining the tools

In [15]:
import random

def get_weather(location: str) -> float:
    return random.randint(20, 30)

fn_by_name = {f.__name__: f for f in [get_weather]}

# Describing the tools

In [16]:
from openai import OpenAI

client = OpenAI()

tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City and country e.g. Bogotá, Colombia"
                }
            },
            "required": [
                "location"
            ],
            "additionalProperties": False
        },
        "strict": True
    }
}]

messages = [{"role": "user", "content": "What is the weather like in Paris today?"}]

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools
)


In [17]:
messages.append(completion.choices[0].message)

In [ ]:
import json

for tool_call in completion.choices[0].message.tool_calls:
    function_name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)

    messages.append({
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": str(fn_by_name[function_name](**args))
    })



In [22]:
messages

[{'role': 'user', 'content': 'What is the weather like in Paris today?'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_uzzW9U8Qa4kEdxnQ6LGPvoiV', function=Function(arguments='{"location":"Paris, France"}', name='get_weather'), type='function')]),
 {'role': 'tool',
  'tool_call_id': 'call_uzzW9U8Qa4kEdxnQ6LGPvoiV',
  'content': '22'}]

In [23]:
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools
)


In [24]:
completion.choices[0].message

ChatCompletionMessage(content='The current temperature in Paris, France is 22°C.', refusal=None, role='assistant', function_call=None, tool_calls=None)